# Part I: Classical Bandit Algorithms

We consider a time-slotted bandit system $(t=1,2, \ldots)$ with three arms. We denote the arm set as $\{1,2,3\}$. Pulling each arm $j(j \in\{1,2,3\})$ will obtain a random reward $r_{j}$, which follows a Bernoulli distribution with mean $\theta_{j}$, i.e., $\operatorname{Bern}\left(\theta_{j}\right)$. Specifically,

$$
r_{j}= \begin{cases}1, & w \cdot p \cdot \theta_{j} \\ 0, & w \cdot p \cdot 1-\theta_{j}\end{cases}
$$

where $\theta_{j}, j \in\{1,2,3\}$ are parameters within $(0,1)$.
Now we run this bandit system for $N(N \gg 3)$ time slots. In each time slot $t$, we choose one and only one arm from these three arms, which we denote as $I(t) \in\{1,2,3\}$. Then we pull the arm $I(t)$ and obtain a random reward $r_{I(t)}$. Our objective is to find an optimal policy to choose an arm $I(t)$ in each time slot $t$ such that the expectation of the aggregated reward over $N$ time slots is maximized, i.e.,

$$
\max _{I(t), t=1, \ldots, N} \mathbb{E}\left[\sum_{t=1}^{N} r_{I(t)}\right]
$$

If we know the values of $\theta_{j}, j \in\{1,2,3\}$, this problem is trivial. Since $r_{I(t)} \sim \operatorname{Bern}\left(\theta_{I(t)}\right)$,

$$
\mathbb{E}\left[\sum_{t=1}^{N} r_{I(t)}\right]=\sum_{t=1}^{N} \mathbb{E}\left[r_{I(t)}\right]=\sum_{t=1}^{N} \theta_{I(t)}
$$

Let $I(t)=I^{*}=\arg \max \theta_{j}$ for $t=1,2, \ldots, N$, then

$$
\max _{I(t), t=1, \ldots, N} \mathbb{E}\left[\sum_{t=1}^{N} r_{I(t)}\right]=N \cdot \theta_{I^{*}}
$$

However, in reality, we do not know the values of $\theta_{j}, j \in\{1,2,3\}$. We need to estimate the values $\theta_{j}, j \in\{1,2,3\}$ via empirical samples, and then make the decisions in each time slot. Next we introduce three classical bandit algorithms: $\epsilon$-greedy, UCB, and TS, respectively.

<img src="picture/greedy_UCB.png" width="50%" align='left'>
<img src="picture/TS.png" width="50%" align='left'>

## Problems 1  
### Question  
Now suppose we obtain the parameters of the Bernoulli distributions from an oracle, which are shown in the following table. Choose $N=5000$ and compute the theoretically maximized expectation of aggregate rewards over $N$ time slots. We call it the oracle value. Note that these parameters $\theta_{j}, j \in \{1,2,3\}$ and oracle values are unknown to all bandit algorithms.
<center>
<table>
  <tr>
    <th>Arm j</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
  </tr>
  <tr>
    <td>θ<sub>j</td>
    <td>0.7</td>
    <td>0.5</td>
    <td>0.4</td>
  </tr>
</table>
</center>

### Solution
Since each arm's parameter is known from the oracle, we need to choose the arm with the largest parameter to maximize the expectation of aggregate rewards over $N$ time slots.

Given $\theta_1 = 0.7, \theta_2 = 0.5, \theta_3 = 0.4$,
we have $\theta_1 > \theta_2 > \theta_3$.
Thus, we choose arm 1 every time.

i.e. $$\forall t, I(t)=I^*=\arg \max\limits_{j\in\{1,2,3\}}\theta_j=1$$
$$\theta_{I(t)} = \theta_1 = 0.7$$

Since $r_{I(t)} \sim \text{Bern}(\theta_{I(t)})$,

$$E(r_{I(t)}) = \theta_{I(t)}$$

The maximum expected value is 
$$\max_{I(t),t=1,2,\cdots,N}\ E\big[\sum_{t=1}^Nr_{I(t)}\big]$$
$$=\max_{I(t),t=1,2,\cdots,N}\ \sum_{t=1}^NE\big[r_{I(t)}\big]$$
$$=N \cdot \theta_{I^*} = 5000 \times 0.7 = 3500$$

Therefore, with the given oracle parameters, the maximum expected value is 3500.


## Problem 2  
### Question
2. Implement classical bandit algorithms with following settings: 
   - $N=5000$
   - $\epsilon$-greedy with $\epsilon \in \{0.1, 0.5, 0.9\}$.
   - UCB with $c \in \{1,5,10\}$.
   - TS with $\left\{(\alpha_1,\beta_1)=(1,1),(\alpha_2,\beta_2)=(1,1),(\alpha_3,\beta_3)=(1,1)\right\}$ and $\left\{(\alpha_1,\beta_1)=(601,401),(\alpha_2,\beta_2)=(401,601),(a3,b3)=(2,3)\right\}$

### Solution  

In [1]:
import numpy as np

class Bandit:
    def __init__(self, theta=[0.7, 0.5, 0.4]):
        self.theta = theta  
        self.n_arms = len(theta)
        self.counts = np.zeros(self.n_arms) 
        self.values = np.zeros(self.n_arms)  
        
    def pull(self, arm):
        return np.random.binomial(1, self.theta[arm])
    
    def update(self, arm, reward):
        self.counts[arm] += 1
        n = self.counts[arm]
        value = self.values[arm]
        self.values[arm] = ((n - 1) / n) * value + (1 / n) * reward

class EpsilonGreedy(Bandit):
    def __init__(self, epsilon, theta=[0.7, 0.5, 0.4]):
        super().__init__(theta)
        self.epsilon = epsilon
        
    def select_arm(self):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        return np.argmax(self.values)
    
    def modify_parameter(self, epsilon):
        self.epsilon = epsilon

class UCB(Bandit):
    def __init__(self, c, theta=[0.7, 0.5, 0.4]):
        super().__init__(theta)
        self.c = c
        
    def select_arm(self):
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        total_counts = sum(self.counts)
        ucb_values = self.values + self.c * np.sqrt(2 * np.log(total_counts) / self.counts)
        return np.argmax(ucb_values)
    
    def modify_parameter(self, c):
        self.c = c

class ThompsonSampling(Bandit):
    def __init__(self, alpha=[1,1,1], beta=[1,1,1], theta=[0.7, 0.5, 0.4]):
        super().__init__(theta)
        self.alpha = np.array(alpha)
        self.beta = np.array(beta)
        
    def select_arm(self):
        samples = [np.random.beta(self.alpha[i], self.beta[i]) for i in range(self.n_arms)]
        return np.argmax(samples)
    
    def update(self, arm, reward):
        super().update(arm, reward)
        self.alpha[arm] += reward
        self.beta[arm] += (1 - reward)

    def modify_parameter(self, alpha, beta):
        self.alpha = np.array(alpha)
        self.beta = np.array(beta)

## Problem 3
Each experiment lasts for N = 5000 time slots, and we run each experiment 200 trials. Results are averaged over these 200 independent trials.  

In [2]:
N = 5000 
num_trials = 200 
experiments = {
    'epsilon_greedy': [0.1, 0.5, 0.9],  
    'ucb': [1, 5, 10],  
    'ts': [
        ([1, 1, 1], [1, 1, 1]), 
        ([601, 401, 2], [401, 601, 3]) 
    ]
}
results_rewards = {
    'epsilon_greedy': [],
    'ucb': [],
    'ts': []
}
results_regrets = {
    'epsilon_greedy': [],
    'ucb': [],
    'ts': []
}

for key in ['epsilon_greedy', 'ucb', 'ts']:
    for value in experiments[key]:
        cumulative_rewards = 0
        cumulative_regrets = 0
        
        for _ in range(num_trials):
            if key == 'epsilon_greedy':
                bandit = EpsilonGreedy(epsilon=value)
            elif key == 'ucb':
                bandit = UCB(c=value)
            elif key == 'ts':
                bandit = ThompsonSampling(alpha=value[0], beta=value[1])
                
            trial_rewards = []
            for t in range(N):
                arm = bandit.select_arm()
                reward = bandit.pull(arm)
                bandit.update(arm, reward)
                cumulative_rewards += reward
                regret = max(bandit.theta) - bandit.theta[arm]
                cumulative_regrets += regret
        results_rewards[key].append(cumulative_rewards / num_trials)
        results_regrets[key].append(cumulative_regrets / num_trials)

print("epsilon = 0.1   reward: ", results_rewards['epsilon_greedy'][0], " regret: ", results_regrets['epsilon_greedy'][0])
print("epsilon = 0.5   reward: ", results_rewards['epsilon_greedy'][1], " regret: ", results_regrets['epsilon_greedy'][1])
print("epsilon = 0.9   reward: ", results_rewards['epsilon_greedy'][2], " regret: ", results_regrets['epsilon_greedy'][2])
print("c = 1          reward: ", results_rewards['ucb'][0], " regret: ", results_regrets['ucb'][0])
print("c = 5          reward: ", results_rewards['ucb'][1], " regret: ", results_regrets['ucb'][1])
print("c = 10         reward: ", results_rewards['ucb'][2], " regret: ", results_regrets['ucb'][2])
print("alpha = [1,1,1] beta = [1,1,1] reward: ", results_rewards['ts'][0], " regret: ", results_regrets['ts'][0])
print("alpha = [601,401,2] beta = [401,601,3] reward: ", results_rewards['ts'][1], " regret: ", results_regrets['ts'][1])

epsilon = 0.1   reward:  3412.44  regret:  86.41450000000351
epsilon = 0.5   reward:  3079.025  regret:  418.2264999999955
epsilon = 0.9   reward:  2753.085  regret:  749.9399999999831
c = 1          reward:  3413.315  regret:  87.61050000003641
c = 5          reward:  2978.395  regret:  519.4099999993863
c = 10         reward:  2829.53  regret:  672.5744999995777
alpha = [1,1,1] beta = [1,1,1] reward:  3483.01  regret:  18.015999999998755
alpha = [601,401,2] beta = [401,601,3] reward:  3490.23  regret:  8.311499999999198
